In [12]:
from langchain.llms.openai import OpenAI
from langchain.chat_models import ChatOpenAI

llm = OpenAI(model_name="gpt-3.5-turbo-1106")
chat = ChatOpenAI(model_name="gpt-4-turbo")

a = llm.predict("open ai api 자바로 호출하는 코드 알아?")
b = chat.predict("open ai api 자바로 호출하는 코드 알아?")

a, b


c:\study\workspace\study_fullstack_gpt\env\Lib\site-packages\langchain\llms\openai.py:216: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
c:\study\workspace\study_fullstack_gpt\env\Lib\site-packages\langchain\llms\openai.py:811: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


('네, OpenAI API를 자바로 호출하는 코드는 아래와 같이 작성할 수 있습니다.\n\n```java\nimport okhttp3.MediaType;\nimport okhttp3.OkHttpClient;\nimport okhttp3.Request;\nimport okhttp3.RequestBody;\nimport okhttp3.Response;\nimport org.json.JSONObject;\n\npublic class OpenAiApi {\n\n    private static final String API_KEY = "YOUR_API_KEY";\n    private static final OkHttpClient client = new OkHttpClient();\n\n    public static void main(String[] args) {\n        String prompt = "Once upon a time";\n        String completion = getCompletion(prompt);\n        System.out.println(completion);\n    }\n\n    public static String getCompletion(String prompt) {\n        String url = "https://api.openai.com/v1/engines/davinci/completions";\n        JSONObject json = new JSONObject()\n                .put("prompt", prompt)\n                .put("max_tokens", 100);\n\n        RequestBody body = RequestBody.create(json.toString(), MediaType.parse("application/json"));\n        Request request = new Request.Builder()\n      